In [172]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [173]:
df = pd.read_parquet('sell_in_pred.parquet')
df.head()

,product_id,periodo,primer_periodo,ultimo_periodo,tn_norm,periodo_dt,mes,quarter,month_in_quarter,year,...,prop_product_yearquarter_brand,prop_product_year_cat1,prop_product_year_cat2,prop_product_year_brand,tn_futuro,tn_prediccion,average_tn,std_dev_tn,tn_futuro_real,tn_prediccion_real
0,20001,201910,201701,201912,0.547254,201910,10,4,1,2019,...,25.615935,0.729053,1.300150,6.721442,-0.190568,-0.537512,1398.344322,298.145460,1504.68856,1401.248846
1,20002,201910,201701,201912,3.193077,201910,10,4,1,2019,...,8.629338,0.924228,1.648214,2.181345,-2.936555,-2.275262,1009.368178,303.834835,1087.30855,1288.232643
2,20003,201910,201701,201912,0.658692,201910,10,4,1,2019,...,9.281088,1.524496,2.028025,2.711313,-0.646717,-0.312087,889.004243,292.036581,892.50129,990.225610
3,20004,201910,201701,201912,1.751307,201910,10,4,1,2019,...,9.138012,1.500995,1.996762,2.669516,-1.901520,-1.964250,671.615383,224.450085,637.90002,623.820140
4,20005,201910,201701,201912,1.615325,201910,10,4,1,2019,...,8.555128,1.405251,1.869395,2.499236,-1.848776,-1.102689,644.200514,218.273222,593.24443,756.095265


In [174]:
df_fut = pd.read_parquet('sell_in_pred_futuro.parquet')
df_fut.head()

,product_id,periodo,primer_periodo,ultimo_periodo,tn_norm,periodo_dt,mes,quarter,month_in_quarter,year,...,prop_product_yearquarter_brand,prop_product_year_cat1,prop_product_year_cat2,prop_product_year_brand,tn_futuro,tn_prediccion,average_tn,std_dev_tn,tn_futuro_real,tn_prediccion_real
0,20001,201912,201701,201912,0.356686,201912,12,4,3,2019,...,24.683874,0.702526,1.252843,6.476876,NaN,-0.337291,1398.344322,298.145460,NaN,1404.126833
1,20002,201912,201701,201912,0.256522,201912,12,4,3,2019,...,4.739874,0.507655,0.905322,1.198157,NaN,-0.049735,1009.368178,303.834835,NaN,1072.197335
2,20003,201912,201701,201912,0.011975,201912,12,4,3,2019,...,7.660107,1.258237,1.673822,2.237771,NaN,-0.318828,889.004243,292.036581,NaN,799.391806
3,20004,201912,201701,201912,-0.150213,201912,12,4,3,2019,...,5.474930,0.899303,1.196336,1.599409,NaN,0.037844,671.615383,224.450085,NaN,646.394029
4,20005,201912,201701,201912,-0.233451,201912,12,4,3,2019,...,5.091663,0.836348,1.112587,1.487444,NaN,0.125804,644.200514,218.273222,NaN,620.703991


In [175]:
def perdida(real, prediccion):
    total = real.sum()
    perdida =  abs(real - prediccion).sum()
    resultado = perdida / total
    return resultado

In [176]:
df_pred = df[['product_id', 'tn_futuro_real', 'tn_prediccion_real']]
df_pred['error'] = -(df_pred['tn_futuro_real'] - df_pred['tn_prediccion_real'])/df_pred['tn_futuro_real']*100
df_pred['weight'] = df_pred['tn_futuro_real']/df_pred['tn_futuro_real'].sum()
df_pred

,product_id,tn_futuro_real,tn_prediccion_real,error,weight
0,20001,1504.68856,1401.248846,-6.874493,5.983987e-02
1,20002,1087.30855,1288.232643,18.479032,4.324111e-02
2,20003,892.50129,990.225610,10.949488,3.549383e-02
3,20004,637.90002,623.820140,-2.207224,2.536861e-02
4,20005,593.24443,756.095265,27.450883,2.359270e-02
...,...,...,...,...,...
775,21263,0.01270,0.016155,27.208344,5.050655e-07
776,21265,0.05007,0.076090,51.966352,1.991231e-06
777,21266,0.05121,0.084916,65.819545,2.036567e-06
778,21267,0.01569,0.103538,559.898504,6.239747e-07


In [177]:
perdida = perdida(df_pred['tn_futuro_real'], df_pred['tn_prediccion_real'])
perdida

0.31551944352113664

In [179]:
# df_fut = df_fut[['product_id', 'tn_futuro_real', 'tn_prediccion_real']]
# df_fut =df_fut.merge(df_pred, on='product_id', how='left', suffixes=['FUTURO', 'VALIDACION'])
# df_fut['tn_prediccion_real'] = df_fut['tn_prediccion_realFUTURO'] * (1 -df_fut['error']/100)
# df_fut

In [180]:
kaggle =  df_fut[['product_id','tn_prediccion_real']]
kaggle['tn_prediccion_real'] = kaggle['tn_prediccion_real'].clip(lower=0)
kaggle.columns = ['product_id', 'tn']
kaggle.to_csv('kaggle-new.csv', index=False)